In [1]:
import numpy as np
import pandas as pd

In [2]:


from scipy import stats
import statsmodels.api as sm   # regression

import matplotlib.pyplot as plt


C:\Users\sonww\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
sp500 = pd.read_csv('SP500_DailyIndex-1.csv', parse_dates= ['Date'])

In [4]:
sp500.set_index('Date',inplace=True)

In [5]:
sp500.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1960-01-04,59.910000,59.910000,59.910000,59.910000,59.910000,3990000
1960-01-05,60.389999,60.389999,60.389999,60.389999,60.389999,3710000
1960-01-06,60.130001,60.130001,60.130001,60.130001,60.130001,3730000
1960-01-07,59.689999,59.689999,59.689999,59.689999,59.689999,3310000
1960-01-08,59.500000,59.500000,59.500000,59.500000,59.500000,3290000


In [6]:
sp500.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1, inplace=True)

In [7]:
sp500.head()

,Adj Close
Date,
1960-01-04,59.910000
1960-01-05,60.389999
1960-01-06,60.130001
1960-01-07,59.689999
1960-01-08,59.500000


In [8]:
sp500.columns = ['sp500']

In [9]:
sp500['sp500ret'] = (sp500['sp500'] / sp500['sp500'].shift(1))  -1

In [10]:
sp500.dropna(inplace=True)

In [11]:
ff3 = pd.read_csv('FF3Factors_Daily-1.csv', parse_dates= ['Date'])

In [12]:
ff3.set_index('Date', inplace = True)

In [13]:
df = sp500.join(ff3, how= 'inner')

In [14]:
df.head()

,sp500,sp500ret,Mkt-RF,SMB,HML,RF
Date,,,,,,
1960-01-05,60.389999,0.008012,0.78,-0.40,0.54,0.017
1960-01-06,60.130001,-0.004305,-0.47,0.13,0.36,0.017
1960-01-07,59.689999,-0.007318,-0.65,0.37,0.08,0.017
1960-01-08,59.500000,-0.003183,-0.33,0.18,0.10,0.017
1960-01-11,58.770000,-0.012269,-1.15,0.48,0.32,0.017


In [15]:
df['exret'] = df['sp500ret'] - df['RF'] / 100

In [16]:
df.head()

,sp500,sp500ret,Mkt-RF,SMB,HML,RF,exret
Date,,,,,,,
1960-01-05,60.389999,0.008012,0.78,-0.40,0.54,0.017,0.007842
1960-01-06,60.130001,-0.004305,-0.47,0.13,0.36,0.017,-0.004475
1960-01-07,59.689999,-0.007318,-0.65,0.37,0.08,0.017,-0.007488
1960-01-08,59.500000,-0.003183,-0.33,0.18,0.10,0.017,-0.003353
1960-01-11,58.770000,-0.012269,-1.15,0.48,0.32,0.017,-0.012439


In [17]:
cumret = pd.DataFrame((1+df['exret']).resample('M').prod())

In [18]:
cumret.columns = ['monthly_cumret']

In [19]:
cumret.head()

,monthly_cumret
Date,
1960-01-31,0.925220
1960-02-29,1.006350
1960-03-31,0.982703
1960-04-30,0.980507
1960-05-31,1.024057


In [20]:
df['cumprod'] = (1+ df['exret'] / 100)

In [21]:
def cumretfunc(df): 
    v=(df.prod())-1
    return v

In [22]:
# rollingret = pd.rolling_apply(monthlydf, 12, lambda x: np.prod(x))

In [23]:
cumret['rolling12_cumret'] = cumret.rolling(window=12,center=False).apply(lambda x: cumretfunc(x))

In [24]:
cumret.head(15)

,monthly_cumret,rolling12_cumret
Date,,
1960-01-31,0.925220,NaN
1960-02-29,1.006350,NaN
1960-03-31,0.982703,NaN
1960-04-30,0.980507,NaN
1960-05-31,1.024057,NaN
1960-06-30,1.017061,NaN
1960-07-31,0.973862,NaN
1960-08-31,1.024472,NaN
1960-09-30,0.938025,NaN


In [25]:
# test to see if this is right

cumret['monthly_cumret'].iloc[0:12].prod()

0.94477671173497924

In [26]:
len(df)

14598

In [27]:
# delta = .9836065574
# lst=[]
# for i in range(1,10):
#     returns5 = returns4.iloc[0:i]
#     returns5['Index'] = list(range(len(returns5.index)))
#     returns5['Counter'] = i - returns5['Index']
#     returns5['Weighted r'] = (1 - delta)*delta**returns5['Counter']*returns4['SP500 Excess']
#     returns5['Mean'] = returns5['Weighted r'].sum()
#     returns5['Deviation Sq'] = (1 - delta)*delta**returns5['Counter']*(returns4['SP500 Excess'] - returns5['Mean'])**2
#     Volatility = (returns5['Deviation Sq'].sum()*261)**1/2
#     lst.append(Volatility)
# SP_Volatility = pd.DataFrame(lst)
# SP_Volatility